In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 59.3 MB/s eta 0:00:00


In [2]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [4]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [5]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT")
    else:
        print("✅ Function is BALANCED")


In [6]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)



=== Constant Oracle (f(x)=0) ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
✅ Function is CONSTANT

=== Balanced Oracle (Parity) ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'111': 1024}
✅ Function is BALANCED


Task 1

In [7]:
# ---------- TASK 1: Custom Balanced Oracle ----------
from qiskit import QuantumCircuit

def oracle_balanced_half(qc, inputs, ancilla):
    """
    Custom balanced oracle:
    Flips ancilla if the first two input qubits are both 1 (x0 AND x1).
    Balanced: flips for half of all possible inputs.
    """
    # Use Toffoli (ccx) to flip ancilla when x0 AND x1 = 1
    qc.ccx(inputs[0], inputs[1], ancilla)


# Test the new oracle
if __name__ == "__main__":
    n = 3
    qc_custom = deutsch_jozsa_circuit(
        n, oracle_balanced_half, list(range(n)), n
    )
    print(qc_custom.draw(fold=-1))
    run_dj(qc_custom)

     ┌───┐          ┌───┐┌─┐   
q_0: ┤ H ├───────■──┤ H ├┤M├───
     ├───┤       │  ├───┤└╥┘┌─┐
q_1: ┤ H ├───────■──┤ H ├─╫─┤M├
     ├───┤┌───┐  │  └┬─┬┘ ║ └╥┘
q_2: ┤ H ├┤ H ├──┼───┤M├──╫──╫─
     ├───┤├───┤┌─┴─┐ └╥┘  ║  ║ 
q_3: ┤ X ├┤ H ├┤ X ├──╫───╫──╫─
     └───┘└───┘└───┘  ║   ║  ║ 
c: 3/═════════════════╩═══╩══╩═
                      2   0  1 
Measurement counts: {'011': 246, '010': 257, '000': 245, '001': 276}
✅ Function is BALANCED


Task 2

In [8]:
# ---------- TASK 2: Vary Number of Input Qubits ----------
for n in [2, 4, 5]:
    print(f"\n=== Balanced Oracle with n={n} inputs ===")
    qc_balanced_n = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced_n.draw(fold=-1))
    run_dj(qc_balanced_n)


=== Balanced Oracle with n=2 inputs ===
     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 
Measurement counts: {'11': 1024}
✅ Function is BALANCED

=== Balanced Oracle with n=4 inputs ===
     ┌───┐          ┌───┐     ┌─┐                   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐           
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───────────
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘     ┌─┐   
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫──────┤M├───
     ├───┤       │    │    │   ║ └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────┼────┼───╫───■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ ┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫─┤ X ├─╫───────╫──╫─

Task 3

In [9]:
# ---------- TASK 3: Noise Simulation ----------
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer import AerSimulator

# Build a simple noise model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01, 1), ["u3", "u2"])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.02, 2), ["cx"])

def run_dj_noisy(qc):
    """Run DJ circuit on noisy AerSimulator"""
    noisy_sim = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, noisy_sim)
    job = noisy_sim.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()
    print("Noisy counts:", counts)
    plot_histogram(counts)
    plt.show()

# Example run with noise
if __name__ == "__main__":
    n = 3
    qc_balanced_noisy = deutsch_jozsa_circuit(
        n, oracle_balanced_half, list(range(n)), n
    )
    print(qc_balanced_noisy.draw(fold=-1))
    run_dj_noisy(qc_balanced_noisy)

     ┌───┐          ┌───┐┌─┐   
q_0: ┤ H ├───────■──┤ H ├┤M├───
     ├───┤       │  ├───┤└╥┘┌─┐
q_1: ┤ H ├───────■──┤ H ├─╫─┤M├
     ├───┤┌───┐  │  └┬─┬┘ ║ └╥┘
q_2: ┤ H ├┤ H ├──┼───┤M├──╫──╫─
     ├───┤├───┤┌─┴─┐ └╥┘  ║  ║ 
q_3: ┤ X ├┤ H ├┤ X ├──╫───╫──╫─
     └───┘└───┘└───┘  ║   ║  ║ 
c: 3/═════════════════╩═══╩══╩═
                      2   0  1 
Noisy counts: {'001': 251, '011': 248, '010': 265, '000': 260}


Task 4

In [10]:
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Choose number of input qubits
n = 3  # change as desired (number of input qubits)

# --- Example 1: constant oracle f(x)=0 (should return all-zeros measurement)
qc_const = deutsch_jozsa_circuit(n, oracle_constant, n, 0)  # note: deutsch_jozsa_circuit expects ancilla index or similar per your definition
print("=== Constant oracle (f(x)=0) circuit ===")
print(qc_const.draw(fold=-1))

# --- Example 2: balanced oracle (parity) ---
inputs = list(range(n))
ancilla = n  # ancilla is the (n)-th qubit (0-indexed)
qc_balanced = deutsch_jozsa_circuit(n, oracle_balanced_parity, inputs, ancilla)
print("\n=== Balanced oracle (parity) circuit ===")
print(qc_balanced.draw(fold=-1))

# --- Run function (local Aer fallback) ---
def run_on_aer(qc, shots=1024):
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    return counts

print("\nRunning constant oracle on AerSimulator...")
counts_const = run_on_aer(qc_const)
print("Counts (constant):", counts_const)
plot_histogram(counts_const)
plt.show()

print("\nRunning balanced (parity) oracle on AerSimulator...")
counts_balanced = run_on_aer(qc_balanced)
print("Counts (balanced):", counts_balanced)
plot_histogram(counts_balanced)
plt.show()




=== Constant oracle (f(x)=0) circuit ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 

=== Balanced oracle (parity) circuit ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 

Running constant oracle on AerSimulator...
Counts (constant): {'000': 1024}

Running balanced (parity) oracle on AerSimulator...
Counts (balanced)

Task 5

In [11]:
# ---------- TASK 5: ORACLE UNITARY ANALYSIS ----------

from qiskit.quantum_info import Operator

qc_oracle = QuantumCircuit(4)  # 3 inputs + 1 ancilla
oracle_balanced_half(qc_oracle, [0,1,2], 3)
print("Oracle gate definition:")
print(qc_oracle.to_gate().definition)

# Display its unitary
unitary = Operator(qc_oracle)
print("\nOracle unitary matrix:")
print(unitary.data)

Oracle gate definition:
          
q_0: ──■──
       │  
q_1: ──■──
       │  
q_2: ──┼──
     ┌─┴─┐
q_3: ┤ X ├
     └───┘

Oracle unitary matrix:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0